# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in a specified format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
  

In [1]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import base64
import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

import pdb

dotenv.load_dotenv()

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'selenium       {selenium.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
selenium       4.20.0
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [3]:
obb

OpenBB Platform v4.1.7

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.0.4
    - crypto@1.1.5
    - currency@1.1.5
    - derivatives@1.1.5
    - econometrics@1.1.5
    - economy@1.1.5
    - equity@1.1.5
    - etf@1.1.5
    - fixedincome@1.1.5
    - index@1.1.5
    - news@1.1.5
    - quantitative@1.1.5
    - regulators@1.1.5
    - technical@1.1.6

    - alpha_vantage@1.1.5
    - benzinga@1.1.5
    - biztoc@1.1.5
    - cboe@1.1.5
    - ecb@1.1.5
    - federal_reserve@1.1.5
    - finra@1.1.5
    - finviz@1.0.4
    - fmp@1.1.5
    - fred@1.1.5
    - government_us@1.1.5
    - intrinio@1.1.5
    - nasdaq@1.1.6
    - oecd@1.1.5
    - polygon@1.1.5
    - sec@1.1.5
    - seeking_alpha@1.1.5
    - stockgrid@1.1.5
    - tiingo@1.1.5
    - tmx@1.0.2
 

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [5]:
obb.user

UserSettings

id: 06642cc4-d6de-7a78-8000-0681e68a9e66
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'benzinga_api_key': None, 'alpha_vantage_api_key': SecretStr('**********'), 'tradier_api_key': None, 'intrinio_api_key': None, 'tiingo_token': SecretStr('**********'), 'fred_api_key': SecretStr('**********'), 'biztoc_api_key': SecretStr('**********'), 'tradier_account_type': None, 'nasdaq_api_key': None, 'polygon_api_key': SecretStr('**********'), 'tradingeconomics_api_key': None, 'fmp_api_key': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot

In [6]:
obb.account

/account
    login
    logout
    save
    refresh
    

In [7]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()

In [8]:
obb.equity

/equity
    /calendar
    /compare
    /darkpool
    /discovery
    /estimates
    /fundamental
    market_snapshots
    /ownership
    /price
    profile
    screener
    search
    /shorts
    

In [9]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [10]:
symbol = "MRK"
company = "Merck"

In [11]:
obj = obb.equity.price.performance(symbol)
obj


OBBject

id: 06642cc5-64ec-7386-8000-f3c5ed6b4487
results: [{'symbol': 'MRK', 'one_day': -0.0059, 'wtd': None, 'one_week': 0.01350000...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [12]:
obj.results


[FinvizPricePerformanceData(symbol=MRK, one_day=-0.0059, wtd=None, one_week=0.013500000000000002, mtd=None, one_month=0.0282, qtd=None, three_month=0.0315, six_month=0.26280000000000003, ytd=0.1859, one_year=0.0999, two_year=None, three_year=None, four_year=None, five_year=None, ten_year=None, max=None, volatility_week=0.0151, volatility_month=0.0147, price=129.29, volume=7053092.0, average_volume=8170000.0, relative_volume=0.86, analyst_recommendation=None, analyst_score=1.48)]

In [13]:
obj.dict()


{'id': '06642cc5-64ec-7386-8000-f3c5ed6b4487',
 'results': [{'symbol': 'MRK',
   'one_day': -0.0059,
   'wtd': None,
   'one_week': 0.013500000000000002,
   'mtd': None,
   'one_month': 0.0282,
   'qtd': None,
   'three_month': 0.0315,
   'six_month': 0.26280000000000003,
   'ytd': 0.1859,
   'one_year': 0.0999,
   'two_year': None,
   'three_year': None,
   'four_year': None,
   'five_year': None,
   'ten_year': None,
   'max': None,
   'volatility_week': 0.0151,
   'volatility_month': 0.0147,
   'price': 129.29,
   'volume': 7053092.0,
   'average_volume': 8170000.0,
   'relative_volume': 0.86,
   'analyst_recommendation': None,
   'analyst_score': 1.48}],
 'provider': 'finviz',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {}},
   'duration': 220496958,
   'route': '/equity/price/performance',
   'timestamp': datetime.datetime(2024, 5, 13, 22, 28, 38, 

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 128.51,
   'bid_size': 800,
   'ask': 129.25,
   'ask_size': 800,
   'last_price': 129.29,
   'open': 129.51,
   'high': 129.71,
   'low': 128.02,
   'volume': 7053004,
   'prev_close': 130.06,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 126.5478,
   'ma_200d': 114.32535,
   'volume_average': 8238929.0,
   'volume_average_10d': 6842750.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 332008292,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-13T22:28:41.932510'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-15,117.14,117.740,115.49,116.37,5697236.0,116.3517,70438
2023-05-16,115.88,116.760,115.29,116.08,4278281.0,116.2071,60730
2023-05-17,116.37,116.655,113.48,114.76,7210383.0,114.5498,78904
2023-05-18,114.14,115.070,113.33,114.00,6437144.0,114.0864,75952
2023-05-19,114.33,116.240,114.10,115.49,7647080.0,115.4905,63052
...,...,...,...,...,...,...,...
2024-05-06,127.18,127.730,126.76,127.57,6419935.0,127.4189,80172
2024-05-07,127.10,130.425,127.07,130.38,6415640.0,129.3458,87045
2024-05-08,130.58,131.510,129.33,129.55,6580073.0,129.7913,74021


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-15,413.01,413.4300,410.230,412.22,427865,54289383.0,412.1201
2023-05-16,410.25,412.8150,410.240,411.86,447536,57705495.0,411.3774
2023-05-17,415.23,415.8550,410.635,412.35,585038,86786957.0,413.4302
2023-05-18,419.23,419.6700,414.670,414.90,672492,97177195.0,417.1828
2023-05-19,418.62,420.7200,417.350,420.17,642871,103793317.0,419.0232
...,...,...,...,...,...,...,...
2024-05-06,516.57,516.6100,513.300,513.75,444747,47236717.0,514.6357
2024-05-07,517.14,518.5700,516.450,517.56,432286,50977654.0,517.4968
2024-05-08,517.19,517.7400,515.140,515.26,369150,42012599.0,516.9211


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='nasdaq', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc. - Common Stock')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,56.76,2024-05-13 20:00:01+00:00,56.98,57.180,56.565,1603317,56.62,...,66.15,54.12,58.7556,60.24095,3118783,1.005844e+11,1772100000,4.61,12.31,2024-05-23 12:00:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,74.70,2024-05-13 20:00:01+00:00,76.17,76.400,74.665,3945784,76.11,...,77.05,48.66,71.6642,63.13665,7050885,1.326717e+11,1776060000,2.39,31.26,2024-07-16 04:00:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,198.73,2024-05-13 20:00:02+00:00,198.80,199.850,198.040,6766527,198.77,...,200.94,133.96,192.1660,166.00345,8849426,5.706870e+11,2871670000,16.58,11.99,2024-07-12 12:30:00+00:00
3,MS,Morgan Stanley,NYSE,98.56,2024-05-13 20:00:01+00:00,98.75,98.965,98.420,4162914,98.28,...,98.99,69.42,91.2008,85.73105,8019831,1.601758e+11,1625160000,5.50,17.92,2024-07-16 04:00:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,tags,id,amp_url,publisher
date,,,,,,,,,,
2024-05-08 06:26:57+00:00,Jim Cramer Advises Investors To Brace For Econ...,"Jim Cramer, the host of CNBC’s “Mad Money,” ha...",[{'url': 'https://cdn.benzinga.com/files/image...,https://www.benzinga.com/news/24/05/38691510/j...,"AAPL,BLDR,GOOG,NVDA,MRK,PFE,GOOGL,META",Benzinga Neuro,"News,Global,Economics,Federal Reserve,Markets",nabJYp9Ewiso_ihbgnuIkdcXtn3Ihy-oucZmO2ALnCk,https://www.benzinga.com/amp/content/38691510,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-09 13:00:16+00:00,"Investors Heavily Search Merck & Co., Inc. (MR...","Recently, Zacks.com users have been paying clo...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2271265/inves...,MRK,Zacks Equity Research,None,LThWA5Sh3t7k8BJ8547R71qJY_9Mgkha7WMQAbZTGro,https://www.zacks.com/amp/stock/news/2271265/i...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-10 15:18:00+00:00,Pharma Stock Roundup: PFE DMD Study Patient De...,Pfizer (PFE) reports the death of a participan...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2272311/pharm...,"PFE,MRK,LLY",Kinjel Shah,None,LuH9atBP810MgCmcYvB9qbOEDI5RlsMVZB1l1s1W8D8,https://www.zacks.com/amp/stock/news/2272311/p...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-13 08:06:09+00:00,My Dividend Growth Portfolio: Selling Bio-Phar...,I have shifted my strategy to focus on quality...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4692695-my-di...,"BKNG,TSLA,AMZN,CRM,ZTS,MSCI,ABBV,JNJ,MRK,AVB,O...",Nicholas Ward,None,KC-Ad56Ahj-rJRZbHcBSSkxF_HxZeV2FXNbZJ0q5R70,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-13 15:16:00+00:00,Bristol Myers (BMY) Fails to Meet Goal in Opdi...,Bristol Myers' (BMY) late-stage label-expandin...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2272954/brist...,"AZN,BMY,MRK,LGND",Zacks Equity Research,None,DZ7EFMYgND4Mcq1fh_0B9LZ6EWP-9-JQz_pR6koaUmA,https://www.zacks.com/amp/stock/news/2272954/b...,{'favicon_url': 'https://s3.polygon.io/public/...


# Question Answering Agent

In [21]:
MODEL = "gpt-4o"

MAX_INPUT_TOKENS = 65536     # includes text of all headlines
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0



In [22]:
# utility function to call chatgpt
def get_response(messages, tools, model=MODEL, json_format=False):
    """
    Get a single response from ChatGPT based on a chain of messages.

    Args:
        messages (list): A list of message objects representing the conversation history.
        json_format(boolean): True if JSON response requested. (Last message must express the request for JSON response.)

    Returns:
        dict: A response object containing the generated response from ChatGPT.

    Raises:
        OpenAIError: If there is an error during the API call.

    Example:
        >>> messages = [
        ...     {"role": "system", "content": "You are a helpful assistant."},
        ...     {"role": "user", "content": "What's the weather like today?"},
        ... ]
        >>> response = get_response(messages)
    """

    if tools:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",  # auto is default, but we'll be explicit
            response_format={"type": "json_object"} if json_format else None,
        )
    else:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={"type": "json_object"} if json_format else None,
        )

    return response


In [23]:
client = OpenAI()
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The airspeed velocity of an unladen swallow is a topic that has been popularized by the movie "Monty Python and the Holy Grail." In a more scientific context, one can look at the European Swallow (Hirundo rustica) to get a rough estimate.

Research and ornithological studies suggest that the cruising speed of an unladen European Swallow is roughly 11 meters per second, or about 24 miles per hour.

It's important to remember that flight speed can vary based on factors like wind conditions, the bird's health, and whether it's migrating or simply flying for short distances.


In [25]:
# utility functions to support use of client-side tools when querying openai

def eval_tool(tool_call, verbose=True):
    """
    Given an OpenAI tool_call response,
    evaluates the tool function using the arguments provided by OpenAI,
    and returns the message to send back to OpenAI, including the function return value.

    Args:
        tool_call (object): The OpenAI tool_call response.

    Returns:
        dict: The message to send back to OpenAI, containing the tool_call_id, role, name, and value returned by the tool call.

    """
    function_name = tool_call.function.name
    # look up the function based in global tools on the name
    fn = tools[function_name]['callable']
    # make the tool call's json args into a dict
    kwargs = json.loads(tool_call.function.arguments)
    
    if verbose:
        print(f"{function_name}({str(kwargs)}) -> ", end="")
    # call function with the args and return value
    fn_value = fn(**kwargs)
    if type(fn_value) is list or type(fn_value) is dict:
        fn_value=str(fn_value)
    if verbose:
        output = str(fn_value)
        if len(output) > 100:
            output=output[:100] + "..."
        print(output)

    return {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": tool_call.function.name,
        "content": fn_value,
    }

def get_response_and_eval(messages, tools=[], json_format=False, raw=False, verbose=False):
    """
    Sends a list of messages to OpenAI and returns the response.
    If tool calls are returned, calls all the tools and sends the values back to OpenAI.
    If further tool calls returned, iterates until no more tool calls are returned and
    'stop' is returned as finish_reason, then returns the response.

    Args:
        messages (list): A list of messages to send to OpenAI.
        json_format (boolean): If the final response should be in JSON format.
        raw (boolean): after last tool is called return raw data response that enabled answering the question
        verbose (bool, optional): If True, prints additional information. Defaults to False.

    Returns:
        response: The final response object returned by OpenAI.

    Raises:
        None

    """
    response = get_response(messages, tools=tools, json_format=json_format)
    choice = response.choices[0]
    response_message = choice.message
    finish_reason = choice.finish_reason

    if verbose:
        print(choice)

    while finish_reason != 'stop':
        # Extend conversation with assistant's reply
        messages.append(response_message)
        if finish_reason == 'tool_calls':
            tool_calls = response_message.tool_calls
            if verbose:
                print(tool_calls)
            # Call the tools and add all return values as messages
            for tool in tool_calls:
                messages.append(eval_tool(tool, verbose=True))
            # Get next response
            response = get_response(messages, tools=tools, json_format=json_format)
            choice = response.choices[0]
            response_message = choice.message
            finish_reason = choice.finish_reason
            if verbose:
                output = str(choice)
                output = output[:1000] + "..." if len(output) > 1000 else output
                print(output)
        else:
            print('finish_reason: ', finish_reason)
            break

    if raw:
        # probably want to process that message and return call signature + value
        return messages[-1]
    else:
        return response

def agent_query(user_message, tools, raw=False, verbose=True):
    """
    Send a user message to OpenAI and retrieve the response, calling all tools until done.

    Args:
        user_message (str): The message from the user.
        raw (boolean): after last tool is called return raw data response that enabled answering the question        
        verbose (bool, optional): Display intermediate tool calls and return values. Defaults to False.

    Returns:
        str: The response from the agent.


    Example:
        >>> agent_query("Hello")
        'Hello! How can I assist you today?'
    """

    # add descriptions of available tools to system prompt
    tool_descs = ""
    tools = list(tools.values())
    openai_tools = []
    for v in tools:
        t = v['tooldict']
        openai_tools.append(t)
        tname = t['function']['name']
        tdesc = t['function']['description']
        tool_descs += f"{tname} : {tdesc}"
        example_str = v.get('example_str')
        if example_str:
            tool_descs += f" Usage: {example_str} "
        tool_descs += "\n---\n\n"
    # tool_descs = "\n".join([f"{tool['function']['name']} : {tool['function']['description']}" for tool in tools.values()])
    current_system_prompt = system_prompt + f"""

Available tools delimited by ---:
{tool_descs}
    """
    # print(current_system_prompt)

    messages = [{"role": "system", "content": current_system_prompt},
                {"role": "user", "content": user_message}]
    response = get_response_and_eval(messages, tools=openai_tools, json_format=False, raw=raw, verbose=verbose)
    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\\$")   # escape stuff that is interpreted as latex
    display(Markdown(response_str))



In [81]:
# for the frequent case where we call a function based on a symbol, make a generic tool class
class BB_agent_tool(object):

    # class variable to keep track of all tools
    agent_registry = {}

    def __init__(self, name, description, openapi_path, parameters, example_parameter_values, singular=0):
        self.name = name
        self.description = description
        self.openapi_path = openapi_path
        self.parameters = parameters
        self.example_parameter_values = example_parameter_values
        self.singular = singular
        self.callable = self.get_callable()
        self.tooldict = self.make_tooldict()
        self.example_code = self.make_example_code()
        # store self in agent_registry class variable that keeps track of all instances
        self.agent_registry[name] = self

    def __del__(self):
        del self.agent_registry[self.name]

    def __str__(self):
        return (f"Tool Name:          {self.name}\n"
                f"Description:        {self.description}\n"
                f"OpenAPI Path:       {self.openapi_path}\n"
                f"Parameters:         {self.parameters}\n"
                f"Example Parameters: {self.example_parameter_values}\n"
                f"Singular:           {self.singular}\n"
                f"Tooldict:           {self.tooldict}\n"
                f"Examples:           {self.example_code}\n"
                f"Callable:           \n"
                f"{self.callable.__doc__}\n"
        )

    def __call__(self, symbol):
        """Make the instance callable and perform the default operation of calling the callable function on a symbol."""
        retval = None
        try:
            obj = self.callable(symbol)
            if obj and obj.results:
                d = json.loads(obj.json())
                # singular should be 0 to return the full array, 1 for first element only, -1 for last element
                if self.singular:
                    index = 0 if self.singular == 1 else -1
                    retval = json.dumps(d['results'][index])
                else:
                    retval = json.dumps(d['results'])
        except Exception as exc:
            print(exc)
        return str(retval)

    def get_callable(self):
        """for /api/vi/equity/search, return the openbb object obb.equity.search"""
        op = obb
        for part in self.openapi_path.removeprefix("/api/v1/").split("/"):
            op = op.__getattribute__(part)
        return op

    def make_tooldict(self):
        """the representation of the tool that is shared with openai"""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": {
                    "type": "object",
                    "properties": self.parameters,
                    "required": [k for k in self.parameters.keys()]   # all required for now
                    }
                }
            }

    def make_example_code(self):
        examples = []
        for example in self.example_parameter_values:
            parray = []
            for k, v in example.items():
                # get type 
                ptype = "string"
                if ptype == "string":
                    parray.append(f'{k}="{v}"') 
                else:
                    parray.append(f'{k}={v}') 
            pstr = ", ".join(parray) 

            # run the example
            return_value = self.callable(**example)
            if type(return_value) is list: # only use 3 values if it's a long list
                return_value = str(return_value[:3])
            if type(return_value) is str:  # truncate long strings
                max_str_len = 1000
                return_value = return_value[:max_str_len] + "..." if len(return_value) > max_str_len else return_value
            
            examples.append(f"{self.name}({pstr}) -> {return_value}")
            return "; ".join(examples)

    def make_metadata(self):
        return {'name': self.name,
                  'description': self.description,
                  'openapi_path': self.openapi_path,
                  'callable': self,
                  'parameters': self.parameters,
                  'example_parameters': self.example_parameter_values,
                  'tooldict': self.tooldict,
                  'example_str': self.example_code}



# Example usage
tool = BB_agent_tool(
    name="get_quote_json",
    description="Given a stock symbol, get the latest market data quote for the stock in JSON format.",
    openapi_path="/api/v1/equity/price/quote",
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

print(tool('AAPL'))
tool.make_metadata()


[{"symbol": "AAPL", "asset_type": "stock", "name": "APPLE INC COM", "exchange": null, "bid": 186.11, "bid_size": 6, "bid_exchange": null, "ask": 186.19, "ask_size": 2, "ask_exchange": null, "quote_conditions": null, "quote_indicators": null, "sales_conditions": null, "sequence_number": null, "market_center": null, "participant_timestamp": null, "trf_timestamp": null, "sip_timestamp": null, "last_price": 186.14, "last_tick": "down", "last_size": null, "last_timestamp": "2024-05-13T15:59:59", "open": 185.5, "high": 187.1, "low": 184.62, "close": 186.28, "volume": 72044809, "exchange_volume": null, "prev_close": 183.05, "change": 3.23, "change_percent": 0.017645, "year_high": 199.6199951171875, "year_low": 164.07000732421875, "iv30": 0.19427, "iv30_change": null, "iv30_change_percent": null, "iv30_annual_high": 0.3034000015258789, "hv30_annual_high": 0.28184000015258787, "iv30_annual_low": 0.15277999877929688, "hv30_annual_low": 0.12411399841308594, "iv60_annual_high": 0.27665000915527344

{'name': 'get_quote_json',
 'description': 'Given a stock symbol, get the latest market data quote for the stock in JSON format.',
 'openapi_path': '/api/v1/equity/price/quote',
 'callable': <__main__.BB_agent_tool at 0x3359aeed0>,
 'parameters': {'symbol': {'type': 'string',
   'description': 'The stock symbol.'}},
 'example_parameters': [{'symbol': 'NVDA'}],
 'tooldict': {'type': 'function',
  'function': {'name': 'get_quote_json',
   'description': 'Given a stock symbol, get the latest market data quote for the stock in JSON format.',
   'parameters': {'type': 'object',
    'properties': {'symbol': {'type': 'string',
      'description': 'The stock symbol.'}},
    'required': ['symbol']}}},
 'example_str': 'get_quote_json(symbol="NVDA") -> id=\'06642d4c-eace-751d-8000-31e77dd8a63c\' results=[CboeEquityQuoteData(symbol=NVDA, asset_type=stock, name=NVIDIA CORPORATION COM, exchange=None, bid=902.55, bid_size=1, bid_exchange=None, ask=902.8, ask_size=2, ask_exchange=None, quote_conditio

In [35]:
tools = {}

In [36]:
# make a bespoke tool 

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    return item1_text

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, get item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "example_parameters": [{
        "symbol": "MSFT",
    }],
}

# create tool dict based on metadata
def make_tool_dict(fn_metadata):
    retdict  = {
        "type": "function",
        "function": {
            "name": fn_metadata["name"],
            "description": fn_metadata["description"],
            "parameters": {
                "type": "object",
                "properties": fn_metadata["parameters"],
                "required": [k for k in fn_metadata["parameters"].keys()]
                }
            }
        }
    return retdict

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [37]:
item_1_text = get_10k_item1_from_symbol(symbol)
len(item_1_text)

190265

In [41]:
def create_example_code(fn_metadata):
    example_str = ""
    for example in fn_metadata['example_parameters']:
        if example_str:
            example_str += " ; "
        example_str += f"{fn_metadata['name']}("
        parray = []
        for k, v in example.items():
            # get type 
            ptype = "string"
            if ptype == "string":
                parray.append(f'{k}="{v}"') 
            else:
                parray.append(f'{k}={v}') 
        example_str += ", ".join(parray) 
        example_str += ")" 
        example_str += " -> " 
        return_value = fn_metadata["callable"](**example)
        if type(return_value) is list:
            return_value = str(return_value[:3])
        if type(return_value) is str:
            return_value = return_value[:1000] + "..." if len(return_value) > 1000 else return_value
        example_str += return_value
        return example_str

create_example_code(fn_metadata)


'get_10k_item1_from_symbol(symbol="MSFT") -> Note About Forward-Looking Statements\nThis report includes estimates, projections, statements relating to our business plans, objectives, and expected operating results that are “forward-looking statements” within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. Forward-looking statements may appear throughout this report, including the following sections: “Business” (Part I, Item 1 of this Form 10-K), “Risk Factors” (Part I, Item 1A of this Form 10-K), and “Management’s Discussion and Analysis of Financial Condition and Results of Operations” (Part II, Item 7 of this Form 10-K). These forward-looking statements generally are identified by the words “believe,” “project,” “expect,” “anticipate,” “estimate,” “intend,” “strategy,” “future,” “opportunity,” “plan,” “may,” “should,” “will,” “would,” “will be,” “will continue,” “w

In [42]:
system_prompt = """
Role: You are an AI stock market assistant tasked with providing up-to-date and detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate and concise answers relevant to their questions about individual stocks.

Capabilities: You are given a number of tools as functions. Use as many tools as needed to ensure all information provided is timely, accurate, concise, relevant, and specific to the user's query.

Instructions: First, analyze the query to fully understand the intent of the user, what data they need, and which tools will satisfy their request.
Then, call the most relevant tools in the order necessary to respond accurately to the investor query. 
If the question is not about a specific stock, if no relevant tool is available, or if uncertain or lacking data, say so clearly and suggest alternative resources or data sources that may help.
Only use data obtained from the tools.

Example Interaction:
Investor: What is the current P/E ratio of Apple Inc.?
AI Response: As of the close of trading on March 10, 2024, Apple Inc.'s P/E ratio is 28.45.
"""


In [43]:
tools

{'get_10k_item1_from_symbol': {'name': 'get_10k_item1_from_symbol',
  'description': "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
  'openapi_path': None,
  'callable': <function __main__.get_10k_item1_from_symbol(symbol)>,
  'parameters': {'symbol': {'type': 'string',
    'description': 'The symbol to get the 10-K item 1 for'}},
  'example_parameters': [{'symbol': 'MSFT'}],
  'tooldict': {'type': 'function',
   'function': {'name': 'get_10k_item1_from_symbol',
    'description': "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    'parameters': {'type': 'object',
     'properties': {'symbol': {'type': 'string',
       'description': 'The symbol to get the 10-K item 1 for'}},
     'required': ['symbol']}}}}}

In [44]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(user_message, tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1X2WST7G0jZjiTT3zNVgs1UY', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_1X2WST7G0jZjiTT3zNVgs1UY', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='function')]
get_10k_item1_from_symbol({'symbol': 'MSFT'}) -> Note About Forward-Looking Statements
This report includes estimates, projections, statements relati...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="**Summary of Item 1 from the Latest 10-K Annual Report for Microsoft Corporation (MSFT):**\n\n**General Overview:**\nMicrosoft Corporation is a global technology company founded in 1975. Its mission is to empower every person and organization to achieve more, and

**Summary of Item 1 from the Latest 10-K Annual Report for Microsoft Corporation (MSFT):**

**General Overview:**
Microsoft Corporation is a global technology company founded in 1975. Its mission is to empower every person and organization to achieve more, and it provides a variety of products and services, including software, cloud services, and hardware.

**Core Business Focus:**
Microsoft's strategy revolves around three interconnected ambitions:
1. Reinventing productivity and business processes
2. Building the intelligent cloud and intelligent edge platform
3. Creating more personal computing

**Products and Services:**
- **Software and Services:** Operating systems (Windows), productivity and business processes (Office 365, Microsoft Teams, Dynamics 365), and AI-based solutions.
- **Cloud Services:** Azure, which includes infrastructure as a service (IaaS), platform as a service (PaaS), and AI capabilities.
- **Devices:** PCs, tablets, consoles (Xbox), and other intelligent devices.
- **Other Offerings:** LinkedIn, GitHub, and advertising services.

**Intelligent Cloud and AI:**
Microsoft focuses heavily on cloud computing and AI. Azure offers a range of cloud services, and partnerships, like the one with OpenAI, enhance its AI capabilities. Commitment to responsible AI development is a key principle.

**Research and Innovation:**
The company continues to invest in research and development across areas such as cloud computing, AI, productivity tools, and security. It emphasizes a comprehensive approach to innovation, integrating emerging technologies like AI into its product offerings.

**Corporate Social Responsibility:**
Microsoft is committed to sustainability and has outlined goals to become carbon negative, water positive, and zero waste by 2030. It also focuses on addressing racial injustice, increasing digital skills, and enhancing accessibility.

**Market Strategy and Competition:**
Microsoft faces intense competition from global companies in software, cloud services, and hardware. Its competition includes Apple, Google, Amazon, and IBM. Microsoft focuses on offering integrated and secure solutions to maintain its competitive edge.

**Legal and Regulatory Environment:**
Microsoft engages actively with regulatory bodies and strives to comply with data privacy and cybersecurity laws worldwide.

This summary captures the essence of Microsoft's business strategy, products, innovations, and market positioning as presented in their latest annual report.

In [45]:
# query search using Sec-api
# queryApi = QueryApi(api_key=os.environ['SEC_API_KEY'])

# query = {
#   "query": "ticker:TSLA AND filedAt:[2024-01-01 TO 2024-12-31] AND formType:\"10-K\"",
#   "from": "0",
#   "size": "100",
#   "sort": [{ "filedAt": { "order": "desc" } }]
# }

# response = queryApi.get_filings(query)

# print(response)
# extractorApi = ExtractorApi(os.getenv('SEC_API_KEY'))
# url_10k = 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm'
# item_1_text = extractorApi.get_section(url_10k, "1", "text")
# print(item_1_text)

# 10-K example
# url_10k = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20231231.htm"

# item_1A_text = extractorApi.get_section(url_10k, "1A", "text")
# item_1_text = extractorApi.get_section(url_10k, "1", "text")

# item_7_html = extractorApi.get_section(url_10k, "7", "html")

# 10-Q example
# url_10q = "https://www.sec.gov/Archives/edgar/data/1318605/000095017022006034/tsla-20220331.htm"

# part2_item_1A_text = extractorApi.get_section(url_10q, "part2item1a", "text")

# 8-K example
# url_8k = "https://www.sec.gov/Archives/edgar/data/66600/000149315222016468/form8-k.htm"



# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [46]:
# TODO:

# check system prompt

# check additional prompt improvements

# make conversational ... return multiple symbols and have it clarify

# add start_date and end_date for historical queries

# use assistants API


In [47]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]

"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [48]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [49]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [50]:
# using the openapi.json OpenAPI definition , output a proposed function to call the openbb function
# might be a better way to do this with standard openapi tools to parse the spec. can generate stubs with
# brew install openapi-generator
# openapi-generator generate -i ./openapi.json -g python -o ./openbb_api --skip-validate-spec
# this openapi.json has a bunch of 'examples' keys which are not supported, hence --skip-validate-spec 

# def fn_str(path_str):
#     d = data['paths'][path_str]
#     # operationId = d['get']['operationId']
#     op = path_str.removeprefix("/api/v1/")
#     operation = op.replace('/', '.')    
#     fn_name = op.replace('/', '_')
#     docstring = f"{d['get']['description']}"
#     req_params = ", ".join([p['name'] for p in d['get']['parameters'] if p['required']])
#     opt_params = ", ".join([p['name'] for p in d['get']['parameters'] if not p['required']])

#     retval = f"""
# path:                 {path_str}
# operation:            {operation}
# summary:              {d['get']['summary']}
# description:          {d['get']['description']}
# required parameters:  {req_params}
# optional parameters:  {opt_params}

# def get_{fn_name}_json({req_params}):
#     \"\"\"{docstring} (list)
#     \"\"\"

#     retval = None
#     try:
#         obj = obb.{operation}(symbol)
#         if obj and obj.results:
#             d = json.loads(obj.json())
#             retval = json.dumps(d['results'])
#     except Exception as exc:
#         print(exc)
#     return str(retval)

# def get_{fn_name}_json({req_params}):
#     \"\"\"{docstring} (singleton)
#     \"\"\"

#     retval = None
#     try:
#         obj = obb.{operation}(symbol)
#         if obj and obj.results:
#             d = json.loads(obj.json())
#             retval = json.dumps(d['results'][0])
#     except Exception as exc:
#         print(exc)
#     return str(retval)
# """
#     return retval

# path_str = '/api/v1/equity/search'
# print(fn_str(path_str))

In [51]:
def get_equity_search_symbol(search_str):
    """Given a search string, get the stock symbol of the top company whose name best matches the search string.
    """
    retval = None
    try:
        obj = obb.equity.search(search_str)
        if obj and obj.results:
            d = json.loads(obj.json())
            retval = d['results'][0]
    except Exception as exc:
        print(exc)
    return str(retval)

fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "callable": get_equity_search_symbol,
    "parameters": {
        "search_str": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            }
        },
    "example_parameters": [{
        "search_str": "Broadcom",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [52]:
tools['get_equity_search_symbol']

{'name': 'get_equity_search_symbol',
 'description': 'Given a search string, get the stock symbol of the top company whose name best matches the search string.',
 'openapi_path': '/api/v1/equity/search',
 'callable': <function __main__.get_equity_search_symbol(search_str)>,
 'parameters': {'search_str': {'type': 'string',
   'description': 'The search string to match to the stock symbol.'}},
 'example_parameters': [{'search_str': 'Broadcom'}],
 'example_str': 'get_equity_search_symbol(search_str="Broadcom") -> {\'symbol\': \'AVGO\', \'name\': \'BROADCOM INC COM\', \'dpm_name\': \'Susquehanna Securities, LLC\', \'post_station\': \'4/1\'}',
 'tooldict': {'type': 'function',
  'function': {'name': 'get_equity_search_symbol',
   'description': 'Given a search string, get the stock symbol of the top company whose name best matches the search string.',
   'parameters': {'type': 'object',
    'properties': {'search_str': {'type': 'string',
      'description': 'The search string to match to t

In [53]:
agent_query("What is the stock symbol for Salesforce?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XE24HMfHADBhKwCfA5iMfc8f', function=Function(arguments='{"search_str":"Salesforce"}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_XE24HMfHADBhKwCfA5iMfc8f', function=Function(arguments='{"search_str":"Salesforce"}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'search_str': 'Salesforce'}) -> {'symbol': 'CRM', 'name': 'SALESFORCE INC COM', 'dpm_name': 'Citadel Securities LLC', 'post_station'...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The stock symbol for Salesforce is **CRM**.', role='assistant', function_call=None, tool_calls=None))


The stock symbol for Salesforce is **CRM**.

In [54]:
# agent is able to first get symbol, then get annual report for symbol 
agent_query("What is item 1 from the 10k annual report for Microsoft?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jdHYQ6OtrrvRaOry1SOs3pRe', function=Function(arguments='{"search_str":"Microsoft"}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_jdHYQ6OtrrvRaOry1SOs3pRe', function=Function(arguments='{"search_str":"Microsoft"}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'search_str': 'Microsoft'}) -> {'symbol': 'MSFT', 'name': 'MICROSOFT CORP COM', 'dpm_name': 'Susquehanna Securities, LLC', 'post_st...
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yVWHFH1FQs9Yv9aL1pkG9Azj', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='function')]))
[ChatCom

Item 1 from Microsoft's latest 10-K annual report provides a comprehensive overview of the company and its operations. Below are some key highlights:

### General Overview
- **Mission**: Microsoft's mission is to empower every person and every organization on the planet to achieve more by creating local opportunity, growth, and impact in every country.
- **Technology Focus**: Emphasis on platforms and tools powered by artificial intelligence to support small and large business competitiveness, improve educational and health outcomes, grow public-sector efficiency, and empower human ingenuity.

### Core Products and Services
- **Cloud-based Solutions**: Azure AI, Microsoft Cloud, Microsoft Power Platform.
- **Productivity Suite**: Microsoft 365 (Office 365), Dynamics 365, LinkedIn, Microsoft Viva.
- **Devices and Hardware**: Surface PCs, tablets, gaming consoles (Xbox), intelligent devices.
- **Business Segments**:
  - **Productivity and Business Processes**: Includes Office Commercial and Consumer services, LinkedIn, and Dynamics business solutions.
  - **Intelligent Cloud**: Comprises Azure and other cloud services, server products, and Enterprise Services.
  - **More Personal Computing**: Encompasses Windows, Devices, Gaming (Xbox), and Search & News Advertising.

### Sustainability and Corporate Social Responsibility
- **Sustainability Goals**: Committed to being carbon negative, water positive, and zero waste by 2030.
- **Diversity and Inclusion**: Programs to address racial injustice and inequity, representation increase in leadership, and digital skills training worldwide.
- **Global Workforce**: As of June 30, 2023, Microsoft employed approximately 221,000 people on a full-time basis in various roles across the U.S. and internationally.

### Competitive Landscape
- **Competitors**: Google, Amazon, IBM, Oracle, Apple, Sony, and local application developers, among others.
- **Strategic Initiatives**: Investment in AI across the company, cloud infrastructure, hybrid work environments, and business solutions that enhance productivity, security, and collaboration.

### Research and Development
- **Investment Areas**: Cloud computing, AI, devices, operating systems, productivity tools.
- **Research Operations**: Main facilities in Redmond, Washington, with additional R&D centers globally, focusing on a wide spectrum of technologies.

### Distribution and Sales Channels
- **OEMs**: Pre-installing software on new devices, large partnerships with OEMs like Dell, HP, Lenovo.
- **Direct Sales**: Enterprise Agreements, digital marketplaces, online stores.
- **Distributors and Resellers**: Licensing through partners, value-added resellers, and retail outlets.

### Additional Information
- **Brand and Culture**: Emphasis on growth mindset, employee feedback systems, diversity, and pay equity.
- **Workplace Flexibility**: Hybrid work models, physical and mental wellness programs.

This overview captures Microsoft's strategic positioning, operational focus, and approach to innovation and corporate responsibility. If you need more detailed information or specific sections from the report, please let me know!

In [73]:
tool = BB_agent_tool(
    name="get_quote_json",
    description="Given a stock symbol, get the latest market data quote for the stock in JSON format.",
    openapi_path="/api/v1/equity/price/quote",
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

tool('AAPL')


'[{"symbol": "AAPL", "asset_type": "stock", "name": "APPLE INC COM", "exchange": null, "bid": 186.11, "bid_size": 6, "bid_exchange": null, "ask": 186.19, "ask_size": 2, "ask_exchange": null, "quote_conditions": null, "quote_indicators": null, "sales_conditions": null, "sequence_number": null, "market_center": null, "participant_timestamp": null, "trf_timestamp": null, "sip_timestamp": null, "last_price": 186.14, "last_tick": "down", "last_size": null, "last_timestamp": "2024-05-13T15:59:59", "open": 185.5, "high": 187.1, "low": 184.62, "close": 186.28, "volume": 72044809, "exchange_volume": null, "prev_close": 183.05, "change": 3.23, "change_percent": 0.017645, "year_high": 199.6199951171875, "year_low": 164.07000732421875, "iv30": 0.19427, "iv30_change": null, "iv30_change_percent": null, "iv30_annual_high": 0.3034000015258789, "hv30_annual_high": 0.28184000015258787, "iv30_annual_low": 0.15277999877929688, "hv30_annual_low": 0.12411399841308594, "iv60_annual_high": 0.2766500091552734

In [75]:
tools[tool.name]=tool.make_metadata()


In [76]:
tools

{'get_10k_item1_from_symbol': {'name': 'get_10k_item1_from_symbol',
  'description': "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
  'openapi_path': None,
  'callable': <function __main__.get_10k_item1_from_symbol(symbol)>,
  'parameters': {'symbol': {'type': 'string',
    'description': 'The symbol to get the 10-K item 1 for'}},
  'example_parameters': [{'symbol': 'MSFT'}],
  'tooldict': {'type': 'function',
   'function': {'name': 'get_10k_item1_from_symbol',
    'description': "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    'parameters': {'type': 'object',
     'properties': {'symbol': {'type': 'string',
       'description': 'The symbol to get the 10-K item 1 for'}},
     'required': ['symbol']}}}},
 'get_equity_search_symbol': {'name': 'get_equity_search_symbol',
  'description': 'Given a search string, get the stock symbol of the top company whose name best matches the search string.',
  'openap

In [77]:
agent_query(f"What is the last market quote for symbol {symbol}?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_qLFWhS2OlUaLuJRbNa0o4uh0', function=Function(arguments='{"symbol":"MRK"}', name='get_quote_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_qLFWhS2OlUaLuJRbNa0o4uh0', function=Function(arguments='{"symbol":"MRK"}', name='get_quote_json'), type='function')]
get_quote_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "asset_type": "stock", "name": "MERCK & CO INC COM", "exchange": null, "bid": 127...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='As of May 13, 2024, the latest market quote for Merck & Co., Inc. (symbol: MRK) is as follows:\n\n- **Last Price:** $129.29 (last up tick)\n- **Opening Price:** $129.90\n- **Day High:** $129.71\n- **Day Low:** $128.02\n- **Previous Close:** $130.06\n- **Change:** -$0.77 (-0.59%)\n- **Bid Price:** $

As of May 13, 2024, the latest market quote for Merck & Co., Inc. (symbol: MRK) is as follows:

- **Last Price:** \\$129.29 (last up tick)
- **Opening Price:** \\$129.90
- **Day High:** \\$129.71
- **Day Low:** \\$128.02
- **Previous Close:** \\$130.06
- **Change:** -\\$0.77 (-0.59%)
- **Bid Price:** \\$127.44
- **Ask Price:** \\$130.55
- **Volume:** 7,109,715 shares

Additionally, the 52-week high is \\$133.10, and the 52-week low is \\$99.14.

In [79]:
tool = BB_agent_tool(
    name="get_company_profile_json",
    description="Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    openapi_path='/api/v1/equity/profile',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)
tools[tool.name]=tool.make_metadata()

tool('AAPL')


'[{"symbol": "AAPL", "name": "Apple Inc", "cik": null, "cusip": null, "isin": null, "lei": null, "legal_name": null, "stock_exchange": "NASD", "sic": null, "short_description": null, "long_description": "Apple, Inc. engages in the design, manufacture, and sale of smartphones, personal computers, tablets, wearables and accessories, and other varieties of related services. It operates through the following geographical segments: Americas, Europe, Greater China, Japan, and Rest of Asia Pacific. The Americas segment includes North and South America. The Europe segment consists of European countries, as well as India, the Middle East, and Africa. The Greater China segment comprises China, Hong Kong, and Taiwan. The Rest of Asia Pacific segment includes Australia and Asian countries. Its products and services include iPhone, Mac, iPad, AirPods, Apple TV, Apple Watch, Beats products, AppleCare, iCloud, digital content stores, streaming, and licensing services. The company was founded by Steve

In [80]:
agent_query(f"Can you provide a basic company profile of symbol {symbol}", tools=tools, verbose=False)


get_company_profile_json({'symbol': 'MRK'}) -> 

Exception ignored in: <function BB_agent_tool.__del__ at 0x31d7f8540>
Traceback (most recent call last):
  File "/var/folders/6d/3xz907yn5ylg43s2vlnnzptr0000gn/T/ipykernel_22099/3936278489.py", line 21, in __del__
AttributeError: 'BB_agent_tool' object has no attribute 'tool_name'
Exception ignored in: <function BB_agent_tool.__del__ at 0x3359cb920>
Traceback (most recent call last):
  File "/var/folders/6d/3xz907yn5ylg43s2vlnnzptr0000gn/T/ipykernel_22099/2930527438.py", line 21, in __del__
KeyError: 'get_quote_json'


[{"symbol": "MRK", "name": "Merck & Co Inc", "cik": null, "cusip": null, "isin": null, "lei": null, ...


### Merck & Co., Inc. (Symbol: MRK)

**Business Description:**
Merck & Co., Inc. is a leading health care company focused on providing health solutions through a range of products. The company operates in the following segments:
- **Pharmaceutical:** Includes human health pharmaceutical and vaccine products.
- **Animal Health:** Focuses on discovering, developing, manufacturing, and marketing pharmaceutical and vaccine products for the prevention, treatment, and control of disease in livestock and companion animal species.
- **Other:** Comprises sales from non-reportable segments of healthcare services.

**Basic Details:**
- **Founded:** 1891
- **Headquarters:** Rahway, NJ, USA
- **Sector:** Healthcare
- **Industry Category:** Drug Manufacturers - General
- **Stock Exchange:** NYSE
- **Indexes:** DJIA, S&P 500
- **Employees:** 72,000
- **Market Capitalization:** \\$327.47 billion
- **Shares Outstanding:** 2.53 billion
- **Institutional Ownership:** 78.09%
- **Short Interest:** 18.75 million
- **Earnings Date:** April 25 (Before Market Open)
- **Beta:** 0.42

Merck & Co., Inc. is known for its prescription medicines, vaccines, biologic therapies, animal health, and consumer care products, aiming to enhance the quality of life and health care for people and animals around the world.

In [82]:
tool = BB_agent_tool(
    name="get_equity_shorts_short_interest",
    description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
    openapi_path='/api/v1/equity/shorts/short_interest',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)
tools[tool.name]=tool.make_metadata()

tool('AAPL')


'[{"settlement_date": "2021-07-15", "symbol": "AAPL", "issue_name": "Apple Inc. Common Stock", "market_class": "NNM", "current_short_position": 96355309.0, "previous_short_position": 90213531.0, "avg_daily_volume": 96035157.0, "days_to_cover": 1.0, "change": 6141778.0, "change_pct": 6.81}, {"settlement_date": "2021-07-30", "symbol": "AAPL", "issue_name": "Apple Inc. Common Stock", "market_class": "NNM", "current_short_position": 93114834.0, "previous_short_position": 96355309.0, "avg_daily_volume": 87082127.0, "days_to_cover": 1.07, "change": -3240475.0, "change_pct": -3.36}, {"settlement_date": "2021-08-13", "symbol": "AAPL", "issue_name": "Apple Inc. Common Stock", "market_class": "NNM", "current_short_position": 93105968.0, "previous_short_position": 93114834.0, "avg_daily_volume": 58359210.0, "days_to_cover": 1.6, "change": -8866.0, "change_pct": -0.01}, {"settlement_date": "2021-08-31", "symbol": "AAPL", "issue_name": "Apple Inc. Common Stock", "market_class": "NNM", "current_shor

In [83]:
agent_query(f"Provide latest statistics on short interest for symbol {symbol}", tools=tools, verbose=False)


get_equity_search_symbol({'search_str': 'MRK'}) -> {'symbol': 'FDT', 'name': 'FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US', 'dpm_name': 'Belvedere Tra...
get_equity_search_symbol({'search_str': 'Merck & Co., Inc.'}) -> 
Type -> str

Detail -> query -> multiple items not allowed for 'cboe'
None
get_equity_shorts_short_interest({'symbol': 'MRK'}) -> [{"settlement_date": "2021-07-15", "symbol": "MRK", "issue_name": "Merck & Co., Inc.", "market_class...


Here are the latest short interest statistics for Merck & Co., Inc. (NYSE: MRK):

- **Settlement Date:** April 30, 2024
- **Current Short Position:** 18,754,018 shares
- **Previous Short Position:** 20,429,972 shares
- **Average Daily Volume:** 7,551,651 shares
- **Days to Cover:** 2.48
- **Change:** -1,675,954 shares
- **Change Percentage:** -8.2%

To provide further context, here are additional data points over the recent period:

- **March 15, 2024**
  - **Short Position:** 23,155,920 shares
  - **Average Daily Volume:** 12,659,181 shares
  - **Days to Cover:** 1.83
  - **Change:** +6,551,186 shares
  - **Change Percentage:** +39.45%
  
- **April 15, 2024**
  - **Short Position:** 20,429,972 shares
  - **Average Daily Volume:** 6,246,795 shares
  - **Days to Cover:** 3.27
  - **Change:** +837,750 shares
  - **Change Percentage:** +4.28%

These metrics indicate notable fluctuations in short interest over recent months, with the most recent data showing a significant decrease.

Let me know if you need any further details or additional context!

In [84]:
tool = BB_agent_tool(
    name="get_equity_fundamental_historical_splits",
    description="Given a stock symbol, get the company's historical stock splits in JSON format.",
    openapi_path='/api/v1/equity/fundamental/historical_splits',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)
tools[tool.name]=tool.make_metadata()

tool('AAPL')


'[{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August 31, 20"}, {"date": "2014-06-09", "numerator": 7.0, "denominator": 1.0, "split_ratio": null, "label": "June 09, 14"}, {"date": "2005-02-28", "numerator": 2.0, "denominator": 1.0, "split_ratio": null, "label": "February 28, 05"}, {"date": "2000-06-21", "numerator": 2.0, "denominator": 1.0, "split_ratio": null, "label": "June 21, 00"}, {"date": "1987-06-16", "numerator": 2.0, "denominator": 1.0, "split_ratio": null, "label": "June 16, 87"}]'

In [85]:
agent_query(f"Provide historical split information for symbol AAPL", tools=tools, verbose=False)

get_equity_search_symbol({'search_str': 'Apple'}) -> {'symbol': 'APLE', 'name': 'APPLE HOSPITALITY REIT INC COM NEW', 'dpm_name': 'Susquehanna Securities...
get_equity_search_symbol({'search_str': 'Apple Inc.'}) -> {'symbol': 'AAPL', 'name': 'APPLE INC COM', 'dpm_name': 'Susquehanna Securities, LLC', 'post_station...
get_equity_fundamental_historical_splits({'symbol': 'AAPL'}) -> [{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August ...


Here are the historical stock splits for Apple Inc. (AAPL):

1. **August 31, 2020**: 4-for-1 split
2. **June 09, 2014**: 7-for-1 split
3. **February 28, 2005**: 2-for-1 split
4. **June 21, 2000**: 2-for-1 split
5. **June 16, 1987**: 2-for-1 split

In [87]:
tool = BB_agent_tool(
    name="get_balance_sheet_json",
    description="Given a stock symbol, get the latest balance sheet data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/balance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)
tools[tool.name]=tool.make_metadata()

tool('AAPL')



'{"period_ending": "2023-09-30", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2023-11-03", "accepted_date": "2023-11-02T18:08:27", "reported_currency": "USD", "cash_and_cash_equivalents": 29965000000.0, "short_term_investments": 31590000000.0, "cash_and_short_term_investments": 61555000000.0, "net_receivables": 60985000000.0, "inventory": 6331000000.0, "other_current_assets": 14695000000.0, "total_current_assets": 143566000000.0, "plant_property_equipment_net": 54376000000.0, "goodwill": null, "intangible_assets": null, "goodwill_and_intangible_assets": -17852000000.0, "long_term_investments": 100544000000.0, "tax_assets": 17852000000.0, "other_non_current_assets": 54097000000.0, "non_current_assets": 209017000000.0, "other_assets": null, "total_assets": 352583000000.0, "accounts_payable": 62611000000.0, "short_term_debt": 15807000000.0, "tax_payables": null, "current_deferred_revenue": 8061000000.0, "other_current_liabilities": 58829000000.0, "total_current_liabilities"

In [ ]:
# can give by year
# obb.equity.fundamental.balance(symbol='NVDA', period='annual', fiscal_year='2023', limit=1)
# also growth from prior period obb.equity.fundamental.balance_growth

In [88]:
agent_query(f"what are the latest total assets for symbol {symbol}", tools=tools, verbose=False)

get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


As of December 31, 2023, the latest total assets for Merck & Co., Inc. (symbol: MRK) are \\$106,675,000,000 (USD).

You can view more details from their filing [here](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [89]:
tool = BB_agent_tool(
    name="get_cash_flow_json",
    description="Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/cash',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)
tools[tool.name]=tool.make_metadata()

tool('AAPL')


'{"period_ending": "2023-09-30", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2023-11-03", "accepted_date": "2023-11-02T18:08:27", "reported_currency": "USD", "net_income": 96995000000.0, "depreciation_and_amortization": 11519000000.0, "deferred_income_tax": 5195000000.0, "stock_based_compensation": 10833000000.0, "change_in_working_capital": -6577000000.0, "change_in_account_receivables": -1688000000.0, "change_in_inventory": -1618000000.0, "change_in_account_payable": -1889000000.0, "change_in_other_working_capital": 5195000000.0, "change_in_other_non_cash_items": -7422000000.0, "net_cash_from_operating_activities": 110543000000.0, "purchase_of_property_plant_and_equipment": -10959000000.0, "acquisitions": null, "purchase_of_investment_securities": -29513000000.0, "sale_and_maturity_of_investments": 45514000000.0, "other_investing_activities": -1337000000.0, "net_cash_from_investing_activities": 3705000000.0, "repayment_of_debt": -9901000000.0, "issuance_of_common_equi

In [91]:
agent_query(f"what was the most recent cash flow from operations for stock symbol {symbol}", tools=tools, verbose=False)

get_quote_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "asset_type": "stock", "name": "MERCK & CO INC COM", "exchange": null, "bid": 127...
get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


As of the most recent filing for fiscal year 2023, Merck & Co., Inc. (MRK) reported cash flow from operating activities of \\$13.006 billion.

For more details, you can access the filing [here](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [92]:
tool = BB_agent_tool(
    name="get_income_statement_json",
    description="Given a stock symbol, get the latest income statement data for the company in JSON format",
    openapi_path='/api/v1/equity/fundamental/income',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

tools[tool.name]=tool.make_metadata()

tool('AAPL')


'{"period_ending": "2023-09-30", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2023-11-03", "accepted_date": "2023-11-02T18:08:27", "reported_currency": "USD", "revenue": 383285000000.0, "cost_of_revenue": 214137000000.0, "gross_profit": 169148000000.0, "gross_profit_margin": 0.4413112958, "general_and_admin_expense": null, "research_and_development_expense": 29915000000.0, "selling_and_marketing_expense": null, "selling_general_and_admin_expense": 24932000000.0, "other_expenses": -565000000.0, "total_operating_expenses": 54847000000.0, "cost_and_expenses": 268984000000.0, "interest_income": 3750000000.0, "total_interest_expense": 3933000000.0, "depreciation_and_amortization": 11519000000.0, "ebitda": 125820000000.0, "ebitda_margin": 0.3282674772, "total_operating_income": 114301000000.0, "operating_income_margin": 0.2982141227, "total_other_income_expenses": -565000000.0, "total_pre_tax_income": 113736000000.0, "pre_tax_income_margin": 0.2967400237, "income_tax_expense":

In [93]:
agent_query(f"what was the most recent net income for symbol {symbol}", tools=tools, verbose=False)


get_equity_search_symbol({'search_str': 'Merck'}) -> {'symbol': 'MRK', 'name': 'MERCK & CO INC COM', 'dpm_name': 'Wolverine Trading, LLC', 'post_station'...
get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent net income for Merck & Co., Inc. (symbol: MRK) is \\$365 million for the fiscal year ending on December 31, 2023. You can find more details in their [10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [94]:
tool = BB_agent_tool(
    name="get_fundamental_metrics_json",
    description="Given a stock symbol, get fundamental metrics for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/metrics',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

tools[tool.name]=tool.make_metadata()

tool('AAPL')


'[{"symbol": "AAPL", "market_cap": 2856440000000.0, "pe_ratio": 28.97, "foward_pe": 25.88, "eps": 6.43, "price_to_sales": 7.48, "price_to_book": 38.51, "book_value_per_share": 4.84, "price_to_cash": 42.54, "cash_per_share": 4.38, "price_to_free_cash_flow": 28.03, "debt_to_equity": 1.41, "long_term_debt_to_equity": 1.24, "quick_ratio": 0.99, "current_ratio": 1.04, "gross_margin": 0.4559, "profit_margin": 0.2631, "operating_margin": 0.3098, "return_on_assets": 0.2999, "return_on_investment": 0.6047, "return_on_equity": 1.4725, "payout_ratio": 0.1532, "dividend_yield": null}]'

In [96]:
agent_query(f"what was the most recent PE ratio for stock symbol {symbol}", tools=tools, verbose=False)

get_quote_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "asset_type": "stock", "name": "MERCK & CO INC COM", "exchange": null, "bid": 127...
get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 327470000000.0, "pe_ratio": 143.93, "foward_pe": 13.05, "eps": 0.9,...


As of the most recent data, Merck & Co., Inc. (ticker symbol: MRK) has a P/E ratio of 143.93.

In [98]:
tool = BB_agent_tool(
    name="get_fundamental_ratios_json",
    description="Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/ratios',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

tools[tool.name]=tool.make_metadata()

tool('AAPL')


'{"period_ending": "2023-09-30", "fiscal_period": "FY", "fiscal_year": 2023, "current_ratio": 0.9880116717592975, "quick_ratio": 0.8433121369780053, "cash_ratio": 0.20621713876730807, "days_of_sales_outstanding": 58.07564866874519, "days_of_inventory_outstanding": 10.791292490321615, "operating_cycle": 68.8669411590668, "days_of_payables_outstanding": 106.72146803214763, "cash_conversion_cycle": -37.85452687308083, "gross_profit_margin": 0.4413112957720756, "operating_profit_margin": 0.2982141226502472, "pretax_profit_margin": 0.296740023742124, "net_profit_margin": 0.2530623426432028, "effective_tax_rate": 0.14719174228036858, "return_on_assets": 0.27509834563776475, "return_on_equity": 1.5607601454639075, "return_on_capital_employed": 0.551446146423833, "net_income_per_ebt": 0.8528082577196314, "ebt_per_ebit": 0.9950569111381353, "ebit_per_revenue": 0.2982141226502472, "debt_ratio": 0.34702467220484257, "debt_equity_ratio": 1.9688314613973545, "long_term_debt_to_capitalization": 0.63

In [99]:
agent_query(f"what was the most recent price to sales ratio for stock symbol {symbol}", tools=tools, verbose=False)

get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 327470000000.0, "pe_ratio": 143.93, "foward_pe": 13.05, "eps": 0.9,...


The most recent price-to-sales ratio for Merck & Co., Inc. (MRK) is 5.36.

In [100]:
tool = BB_agent_tool(
    name="get_equity_fundamental_multiples",
    description="Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/multiples',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

tools[tool.name]=tool.make_metadata()

tool('AAPL')


'{"symbol": "AAPL", "revenue_per_share_ttm": 24.7712947596031, "net_income_per_share_ttm": 6.516288351650178, "operating_cash_flow_per_share_ttm": 7.17668657944096, "free_cash_flow_per_share_ttm": 6.615601236309102, "cash_per_share_ttm": 4.358732160030575, "book_value_per_share_ttm": 4.815960891754409, "tangible_book_value_per_share_ttm": 4.815960891754409, "shareholders_equity_per_share_ttm": 4.815960891754409, "interest_debt_per_share_ttm": 6.983837834132683, "market_cap_ttm": 2856436148000.0, "enterprise_value_ttm": 2931446148000.0, "pe_ratio_ttm": 28.5868258044208, "price_to_sales_ratio_ttm": 7.484968537011658, "pocf_ratio_ttm": 25.956267971021045, "pfcf_ratio_ttm": 28.026532324689214, "pb_ratio_ttm": 38.67971609132814, "ptb_ratio_ttm": 38.67971609132814, "ev_to_sales_ttm": 7.6815237760826784, "enterprise_value_over_ebitda_ttm": 22.6141229817402, "ev_to_operating_cash_flow_ttm": 26.513807946600583, "ev_to_free_cash_flow_ttm": 28.762508933564888, "earnings_yield_ttm": 0.034981148548

In [102]:
agent_query(f"what was the most recent revenue per share for stock symbol {symbol}", tools=tools, verbose=False)


get_quote_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "asset_type": "stock", "name": "MERCK & CO INC COM", "exchange": null, "bid": 127...
get_equity_fundamental_multiples({'symbol': 'MRK'}) -> {"symbol": "MRK", "revenue_per_share_ttm": 24.25424397947098, "net_income_per_share_ttm": 0.91038294...


As of the most recent data, Merck & Co., Inc. (MRK) reported a revenue per share (TTM) of \\$24.25.

In [104]:
# tool = BB_agent_tool(
#     name="get_historical_eps",
#     description="Given a stock symbol, get historical earnings per share data for the company in JSON format.",
#     openapi_path='/api/v1/equity/fundamental/historical_eps',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# tools[tool.name]=tool.make_metadata()

# tool('AAPL')


OpenBBError: 
Type -> EmptyDataError

Detail -> No data found.

In [ ]:
# agent_query(f"what was the EPS for {company} in the quarter ended 2022-09-30?", tools=tools, verbose=False)


In [110]:
tool = BB_agent_tool(
    name="get_equity_fundamental_dividend",
    description="Given a stock symbol, get the latest dividend data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/dividends',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

tools[tool.name]=tool.make_metadata()

tool('AAPL')


'[{"ex_dividend_date": "2024-05-10", "amount": 0.25, "label": "May 10, 24", "adj_dividend": 0.25, "record_date": "2024-05-13", "payment_date": "2024-05-16", "declaration_date": "2024-05-02"}, {"ex_dividend_date": "2024-02-09", "amount": 0.24, "label": "February 09, 24", "adj_dividend": 0.24, "record_date": "2024-02-12", "payment_date": "2024-02-15", "declaration_date": "2024-02-01"}, {"ex_dividend_date": "2023-11-10", "amount": 0.24, "label": "November 10, 23", "adj_dividend": 0.24, "record_date": "2023-11-13", "payment_date": "2023-11-16", "declaration_date": "2023-11-02"}, {"ex_dividend_date": "2023-08-11", "amount": 0.24, "label": "August 11, 23", "adj_dividend": 0.24, "record_date": "2023-08-14", "payment_date": "2023-08-17", "declaration_date": "2023-08-03"}, {"ex_dividend_date": "2023-05-12", "amount": 0.24, "label": "May 12, 23", "adj_dividend": 0.24, "record_date": "2023-05-15", "payment_date": "2023-05-18", "declaration_date": "2023-05-04"}, {"ex_dividend_date": "2023-02-10", 

In [111]:
agent_query(f"what was the latest dividend data for stock symbol {symbol}?", tools=tools, verbose=False)


get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


The latest dividend data for Merck & Co., Inc. (MRK) is as follows:

- **Ex-dividend Date:** March 14, 2024
- **Dividend Amount:** \\$0.77 per share
- **Record Date:** March 15, 2024
- **Payment Date:** April 5, 2024
- **Declaration Date:** January 23, 2024

If you need more historical dividend information or any other details, feel free to ask!

In [112]:
agent_query(f"what was the dividend data as of the end of 2022 for stock symbol {symbol}?", tools=tools, verbose=False)


get_equity_search_symbol({'search_str': 'Merck & Co.'}) -> {'symbol': 'MRK', 'name': 'MERCK & CO INC COM', 'dpm_name': 'Wolverine Trading, LLC', 'post_station'...
get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


As of the end of 2022, Merck & Co. Inc. (symbol MRK) had the following dividend data:

1. **Ex-Dividend Date**: December 14, 2022
   - **Amount**: \\$0.73
   - **Record Date**: December 15, 2022
   - **Payment Date**: January 9, 2023
   - **Declaration Date**: November 29, 2022

2. **Ex-Dividend Date**: September 14, 2022
   - **Amount**: \\$0.69
   - **Record Date**: September 15, 2022
   - **Payment Date**: October 7, 2022
   - **Declaration Date**: July 26, 2022

3. **Ex-Dividend Date**: June 14, 2022
   - **Amount**: \\$0.69
   - **Record Date**: June 15, 2022
   - **Payment Date**: July 8, 2022
   - **Declaration Date**: May 24, 2022

4. **Ex-Dividend Date**: March 14, 2022
   - **Amount**: \\$0.69
   - **Record Date**: March 15, 2022
   - **Payment Date**: April 7, 2022
   - **Declaration Date**: January 25, 2022

In [114]:
tool = BB_agent_tool(
    name="get_trailing_dividend_yield_json",
    description="Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    openapi_path='/api/v1/equity/fundamental/trailing_dividend_yield',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

tools[tool.name]=tool.make_metadata()

tool('AAPL')


'[{"date": "2023-05-12", "trailing_dividend_yield": 0.0053891175}, {"date": "2023-05-15", "trailing_dividend_yield": 0.0054047771}, {"date": "2023-05-16", "trailing_dividend_yield": 0.0054047771}, {"date": "2023-05-17", "trailing_dividend_yield": 0.0053853726}, {"date": "2023-05-18", "trailing_dividend_yield": 0.0053127678}, {"date": "2023-05-19", "trailing_dividend_yield": 0.0053094314}, {"date": "2023-05-22", "trailing_dividend_yield": 0.0053386912}, {"date": "2023-05-23", "trailing_dividend_yield": 0.005420844}, {"date": "2023-05-24", "trailing_dividend_yield": 0.0054120112}, {"date": "2023-05-25", "trailing_dividend_yield": 0.0053760333}, {"date": "2023-05-26", "trailing_dividend_yield": 0.0053012598}, {"date": "2023-05-30", "trailing_dividend_yield": 0.0052453469}, {"date": "2023-05-31", "trailing_dividend_yield": 0.0052468265}, {"date": "2023-06-01", "trailing_dividend_yield": 0.0051640846}, {"date": "2023-06-02", "trailing_dividend_yield": 0.0051395413}, {"date": "2023-06-05", "

In [115]:
agent_query(f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?", tools=tools, verbose=False)


get_equity_search_symbol({'search_str': 'Merck'}) -> {'symbol': 'MRK', 'name': 'MERCK & CO INC COM', 'dpm_name': 'Wolverine Trading, LLC', 'post_station'...
get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{"date": "2023-05-12", "trailing_dividend_yield": 0.0242444938}, {"date": "2023-05-15", "trailing_d...


As of the end of 2023, the trailing dividend yield for Merck & Co Inc. (symbol: MRK) was approximately 2.73%.

In [116]:
tool = BB_agent_tool(
    name="get_price_performance_json",
    description="Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    openapi_path='/api/v1/equity/price/performance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

tools[tool.name]=tool.make_metadata()

tool('AAPL')


'[{"symbol": "AAPL", "one_day": 0.0176, "wtd": null, "one_week": 0.025099999999999997, "mtd": null, "one_month": 0.055099999999999996, "qtd": null, "three_month": -0.0046, "six_month": 0.0212, "ytd": -0.0325, "one_year": 0.0721, "two_year": null, "three_year": null, "four_year": null, "five_year": null, "ten_year": null, "max": null, "volatility_week": 0.014499999999999999, "volatility_month": 0.0177, "price": 186.28, "volume": 70338121.0, "average_volume": 63960000.0, "relative_volume": 1.1, "analyst_recommendation": null, "analyst_score": 2.02}]'

In [117]:
agent_query(f"what was the performance for {company} from 1 year ago?", tools=tools, verbose=False)


get_equity_search_symbol({'search_str': 'Merck'}) -> {'symbol': 'MRK', 'name': 'MERCK & CO INC COM', 'dpm_name': 'Wolverine Trading, LLC', 'post_station'...
get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": -0.0059, "wtd": null, "one_week": 0.013500000000000002, "mtd": null, "...


As of now, Merck & Co., Inc. (MRK) has shown a one-year price performance increase of 9.99%. The current price is \\$129.29, with a one-year volatility of 1.47% per month.

In [119]:
# this might exceed token context making it unreliable
tool = BB_agent_tool(
    name="get_etf_equity_exposure_json",
    description="Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    openapi_path='/api/v1/etf/equity_exposure',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

tools[tool.name]=tool.make_metadata()

tool('AAPL')



'[{"equity_symbol": "AAPL", "etf_symbol": "VTSAX", "shares": 461157515.0, "weight": 0.0493, "market_value": 79079290672.2}, {"equity_symbol": "AAPL", "etf_symbol": "VTI", "shares": 461157515.0, "weight": 0.0493, "market_value": 79079290672.2}, {"equity_symbol": "AAPL", "etf_symbol": "VSMPX", "shares": 461157515.0, "weight": 0.0493, "market_value": 79079290672.2}, {"equity_symbol": "AAPL", "etf_symbol": "VITSX", "shares": 461157515.0, "weight": 0.0493, "market_value": 79079290672.2}, {"equity_symbol": "AAPL", "etf_symbol": "VOO", "shares": 365847962.0, "weight": 0.056299999999999996, "market_value": 62735608523.76}, {"equity_symbol": "AAPL", "etf_symbol": "VFINX", "shares": 365847962.0, "weight": 0.056299999999999996, "market_value": 62735608523.76}, {"equity_symbol": "AAPL", "etf_symbol": "VFIAX", "shares": 365847962.0, "weight": 0.056299999999999996, "market_value": 62735608523.76}, {"equity_symbol": "AAPL", "etf_symbol": "SPY", "shares": 170415228.0, "weight": 0.06106424, "market_val

In [121]:
pd.DataFrame(json.loads(tool('MRK'))).sort_values('weight', ascending=False)

,equity_symbol,etf_symbol,shares,weight,market_value
430,MRK,FTXH,11684.0,0.075700,1.519621e+06
77,MRK,QDVG.DE,1117567.0,0.064076,1.466248e+08
78,MRK,IUHE.AS,0.0,0.062564,1.460573e+08
79,MRK,IUHC.L,1121581.0,0.062374,1.453008e+08
13,MRK,XLV,18454434.0,0.062123,2.400184e+09
...,...,...,...,...,...
557,MRK,GXUS,2163.0,NaN,3.629300e+05
660,MRK,GPAL,640.0,NaN,6.631040e+04
671,MRK,GDEF,397.0,NaN,5.170131e+04
678,MRK,EUNU.DE,50000.0,NaN,3.868500e+04


In [122]:
agent_query(f"which ETF has the highest weight in {company} ?", tools=tools, verbose=False)


get_equity_search_symbol({'search_str': 'Merck'}) -> {'symbol': 'MRK', 'name': 'MERCK & CO INC COM', 'dpm_name': 'Wolverine Trading, LLC', 'post_station'...
get_etf_equity_exposure_json({'symbol': 'MRK'}) -> [{"equity_symbol": "MRK", "etf_symbol": "VTSAX", "shares": 79663326.0, "weight": 0.0066, "market_val...


The ETF with the highest weight in Merck (MRK) is the Health Care Select Sector SPDR Fund (XLV), with a weight of 6.21%. Here are the details:

- **ETF Symbol**: XLV
- **Weight in Merck (MRK)**: 6.21%
- **Market Value of MRK in XLV**: \\$2,400,183,686.04

This indicates that Merck constitutes 6.21% of the holdings in the XLV ETF.

In [123]:
tools

{'get_10k_item1_from_symbol': {'name': 'get_10k_item1_from_symbol',
  'description': "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
  'openapi_path': None,
  'callable': <function __main__.get_10k_item1_from_symbol(symbol)>,
  'parameters': {'symbol': {'type': 'string',
    'description': 'The symbol to get the 10-K item 1 for'}},
  'example_parameters': [{'symbol': 'MSFT'}],
  'tooldict': {'type': 'function',
   'function': {'name': 'get_10k_item1_from_symbol',
    'description': "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    'parameters': {'type': 'object',
     'properties': {'symbol': {'type': 'string',
       'description': 'The symbol to get the 10-K item 1 for'}},
     'required': ['symbol']}}}},
 'get_equity_search_symbol': {'name': 'get_equity_search_symbol',
  'description': 'Given a search string, get the stock symbol of the top company whose name best matches the search string.',
  'openap

In [125]:
[{t['description']} for t in tools.values()]

[{"Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing."},
 {'Given a search string, get the stock symbol of the top company whose name best matches the search string.'},
 {'Given a stock symbol, get the latest market data quote for the stock in JSON format.'},
 {'Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format'},
 {'Given a stock symbol, get data on short volume and days to cover in JSON format.'},
 {"Given a stock symbol, get the company's historical stock splits in JSON format."},
 {'Given a stock symbol, get the latest balance sheet data for the company in JSON format.'},
 {'Given a stock symbol, get the latest cash flow statement data for the company in JSON format.'},
 {'Given a stock symbol, get the latest income statement data for the company in JSON format'},
 {'Given a stock symbol, get fundamental metrics for the company in JSON format.'},
 {'Given a stock symbo